# Alpaca Stocks for Project 2 - Chip and Dip? The Chip Factor? Chip Gaines (pun)?

In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import csv
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Load .env environment variables
project_folder = os.path.expanduser('~/Desktop/project2')  # adjust yours as appropriate
load_dotenv((os.path.join(project_folder, 'SAMPLE.env')))

True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("alpaca_api_key")
alpaca_secret_key = os.getenv("alpaca_secret_key")

In [4]:
# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'NoneType'>
Alpaca Secret Key type: <class 'NoneType'>


In [5]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

ValueError: ('Key ID must be given to access Alpaca trade API', ' (env: APCA_API_KEY_ID)')

In [ ]:
# Format the current date as ISO format
today = pd.Timestamp("2021-09-28", tz="America/New_York").isoformat()

In [ ]:
# Set the tickers for the stocks you have chosen
# We are selecting the following 15 (note to group that 14 currently work, but added in VOO)
tickers = ["TSM", "QCOM", "VALE", "AMD", "BHP", "RIO", "FCX", "INTC", "MSFT", "DDD", "NVDA", "TSLA", "AMAT", "F", "VOO"]

In [ ]:
# Set timeframe to one day ('1D') for the Alpaca API
timeframe = "1D"

In [ ]:
# Get current closing prices for above stocks
df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start = today,
    end = today,
    limit = 500
).df

# Display sample data
# Auto formats in alphabetical order
df_portfolio

In [ ]:
# The farthest Alpaca will go back is 4 months for one pull
# Format start and end dates as ISO format for one year period
start = pd.Timestamp("2021-05-28", tz="America/New_York").isoformat()
end = pd.Timestamp("2021-09-28", tz="America/New_York").isoformat()

In [ ]:
# Get closing prices for chosen stocks from start to end date of choice and set limit to 1000
df_portfolio_months = alpaca.get_barset(
    tickers,
    timeframe,
    start = start,
    end = end,
    limit = 1000
).df

# Display sample data
df_portfolio_months.head(5)

In [ ]:
# Clean the data and drop all columns except for closing
df_portfolio_months_clean = df_portfolio_months.drop(['open', 'low', 'high'], axis=1, level=1)

# Display sample data
df_portfolio_months_clean.head(5)


In [ ]:
# The farthest Alpaca will go back is 4 months for one pull, so we repeat process below
# Format start and end dates as ISO format for four month period
# Use the start date above as the end date below

start_2 = pd.Timestamp("2021-01-01", tz="America/New_York").isoformat()
end_2 = pd.Timestamp("2021-05-27", tz="America/New_York").isoformat()

df_portfolio_months_2 = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_2,
    end = end_2,
    limit = 1000
).df

# Clean the data and drop all columns except for closing
df_portfolio_months_2_clean = df_portfolio_months_2.drop(['open', 'low', 'high'], axis=1, level=1)

# Display sample data
df_portfolio_months_2_clean.head(5)

In [ ]:
# The farthest Alpaca will go back is 4 months for one pull, so we repeat process below
# Format start and end dates as ISO format for four month period
# Use the start date above as the end date below

start_3 = pd.Timestamp("2020-09-28", tz="America/New_York").isoformat()
end_3 = pd.Timestamp("2021-01-01", tz="America/New_York").isoformat()

df_portfolio_months_3 = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_3,
    end = end_3,
    limit = 1000
).df

# Clean the data and drop all columns except for closing
df_portfolio_months_3_clean = df_portfolio_months_3.drop(['open', 'low', 'high'], axis=1, level=1)

# Display sample data
df_portfolio_months_3_clean.head(5)

In [ ]:
#Combine all the cleaned data into one table by using concat
merged_portfolio = pd.concat([df_portfolio_months_3_clean, df_portfolio_months_2_clean, df_portfolio_months_clean])

# Display the head and tail of the newly merged portfolio
merged_portfolio.head()

In [ ]:
# For some reason I am only ever able to show head and tails of a table in two separate code blocks
# So here is the display of the tail of the newly merged portfolio
merged_portfolio.tail()

In [ ]:
# Send to CSV file

merged_portfolio.to_csv('cleaned_stock_data.csv')

In [ ]:
# Graph your stocks closing prices over the course of the last year by using the data you just merged into one portfolio
# We are graphing the data to get a visualization over the stock closing price performance over the last year

# First remove the volume column from the portfolio to just plot the closing prices
merged_portfolio_close_dates = merged_portfolio.drop(['volume'], axis=1, level=1)

merged_portfolio_close_dates.plot()
plt.title('Full Calendar Year of Closing Prices for Chosen Stocks')
plt.ylabel('Closing Prices')
plt.xlabel('Date')
plt.legend(loc="lower right", bbox_to_anchor=(1.4, -0.19), ncol=1)

In [ ]:
# Graph your stocks daily volume over the course of the last year by using the data you just merged into one portfolio
# We are graphing the data to get a visualization over the changes in volume over the last year

# First remove the close column from the portfolio to just plot the volume
merged_portfolio_volumes = merged_portfolio.drop(['close'], axis=1, level=1)

merged_portfolio_volumes.plot()
plt.title('Full Calendar Year of Volume for Chosen Stocks')
plt.ylabel('Volume')
plt.xlabel('Date')
plt.legend(loc="lower right", bbox_to_anchor=(1.4, -0.19), ncol=1)

In [ ]:
# Create a Lagged Volume column that shifts the volume of each stock back by one day

# Since there are multiple columns with the name volume, we must create a loop 
for widget in merged_portfolio.columns.levels[0]:
    merged_portfolio.loc[:,(widget, 'lagged volume')] = merged_portfolio[(widget, 'volume')].shift(1)

# Then we sort the columns to show within each stock
merged_portfolio = merged_portfolio.sort_index(axis=1, level=0)

#Drop NA values
merged_portfolio.dropna(inplace=True)

#Display portfolio 
merged_portfolio.head(3)

In [ ]:
# Create a Daily Stock Return column that will calculate the daily percent change of the closing stock prices

# Since there are multiple columns with the name close, we must create a loop 

for stock in merged_portfolio.columns.levels[0]:
    merged_portfolio.loc[:,(stock, 'daily return')] = merged_portfolio[(stock, 'close')].pct_change()

# Then we sort the columns to show within each stock
merged_portfolio = merged_portfolio.sort_index(axis=1, level=0)

#Display portfolio 
merged_portfolio.tail(3)

In [ ]:
# Create a Stock Volatility column that will calculate the standard deviation of the closing stock prices

# Since there are multiple columns with the name close, we must create a loop 

for volatility in merged_portfolio.columns.levels[0]:
    merged_portfolio.loc[:,(volatility, 'stock volatility')] = merged_portfolio[(volatility, 'close')].pct_change().rolling(window=200).std()

# Then we sort the columns to show within each stock
merged_portfolio = merged_portfolio.sort_index(axis=1, level=0)

#Display portfolio 
merged_portfolio.tail(3)

In [ ]:
# Graph your stock volatility over the course of the last year by using the data you calculated above
# We are graphing the data to get a visualization over the stock volatility over the last year

# First remove the close, volume, lagged volume, and daily return columns from the portfolio to just plot the volume

merged_portfolio_stock_volatility = merged_portfolio.drop(['close', 'volume', 'lagged volume', 'daily return'], axis=1, level=1)

merged_portfolio_stock_volatility.plot()
plt.title('Stock Volatility for Chosen Stocks')
plt.ylabel('Volatility')
plt.xlabel('Date')
plt.legend(loc="lower right", bbox_to_anchor=(1.53, -0.19), ncol=1)

In [ ]:
# Creating a correlation table of the daily stock return, lagged volume, and stock volatility

merged_portfolio_corr = merged_portfolio.drop(['close', 'volume'], axis=1, level=1)

merged_portfolio_corr.corr()

In [ ]:
# Original idea to calculate correlation 

# merged_portfolio_TSM_corr = merged_portfolio.drop(["QCOM", "VALE", "AMD", "BHP", "RIO", "FCX", "INTC", "MSFT", "DDD", "NVDA", "TSLA", "AMAT", "F", "VOO"], axis=1, level=0)

# merged_portfolio_TSM_corr.corr()

In [ ]:
# Below we start running Bollinger Bands to determine return percentage

In [ ]:
merged_portfolio_daily_returns = merged_portfolio.drop(['close', 'volume', 'lagged volume', 'stock volatility'], axis=1, level=1)

merged_portfolio_daily_returns.dropna(inplace=True)

merged_portfolio_daily_returns.head(1)

In [ ]:
# Original strategy to get the covariance of each stock in portfolio

# Below calculates the covariance of TSM to the VOO

merged_portfolio_TSM_cov = merged_portfolio_daily_returns['TSM']['daily return'].cov(merged_portfolio_daily_returns['VOO']['daily return'])

merged_portfolio_TSM_cov



In [ ]:
# Creating a loop to get all the stock covariances to the VOO at once in the order below

stock_ticker_list = ["TSM", "QCOM", "VALE", "AMD", "BHP", "RIO", "FCX", "INTC", "MSFT", "DDD", "NVDA", "TSLA", "AMAT", "F"]

for stock in stock_ticker_list:
    stock_covariance_list = []
    stockcov = merged_portfolio_daily_returns[stock]['daily return'].cov(merged_portfolio_daily_returns['VOO']['daily return'])
    stock_covariance_list+=[stockcov]
    print(stock_covariance_list)
   

In [ ]:
# Create the variance calculation of all the daily returns vs. VOO

VOO_variance = merged_portfolio_daily_returns['VOO']['daily return'].var()

print(VOO_variance)


In [ ]:
# trying to create a loop that divided the covariance of each stock by the variance of VOO, both of which have been calculated above already

for covariance in stock_covariance_list:
    stock_beta_list = []
    stockbetcovarlist / VOO_variance
    